In [43]:
import pandas as pd
import numpy as np
import pathlib as pl
import sklearn
import requests
from bs4 import BeautifulSoup
from time import sleep

In [44]:
def get_path():
    path = './links_sentfin_old.csv'
    df = pd.read_csv(path, encoding = "ISO-8859-1")
    return df

Convert dict column to sentiment and drop those with more than one sentiment

In [45]:
# import ast

# to_drop = []
# for i in range(len(df.index)):
#     aux = ast.literal_eval(df.iloc[i]['Decisions'])
#     df.Decisions[i]= list(aux.values())[0]

#     if len(aux) > 1: # and len(set(list(aux.values()))) > 1
#         to_drop.append(i)
# len(to_drop)

In [46]:
# df = df.drop(to_drop)
# df = df.reset_index(drop=True)
# df

In [47]:
def arrange(df):
    sources = []

    from urllib.parse import urlparse

    for i in range(len(df.index)):
        domain = urlparse(df['links'][i]).netloc
        sources.append(domain)

    df['source'] = pd.Series(sources)
    del sources
    df.source.value_counts().sort_values()
    df = df[df['source'].str.contains("economictimes")]

    df = df.reset_index(drop=True)
    # print(len(df.index))
    return df
# df

In [48]:
# df.source.value_counts().sort_values().plot(kind = 'barh')
# pd.options.display.max_rows = 4000
df.source.value_counts().sort_values()

economictimes.indiatimes.com     3
m.economictimes.com             17
Name: source, dtype: int64

In [49]:
# df = df[df['source'].str.contains("reuters")] #filtering reuters

df = df[df['source'].str.contains("economictimes")]

df = df.reset_index(drop=True)

In [50]:
len(df)

20

In [51]:
def count(start,end,df):
    # start = 0 
    # end = 20 # creo que ya llegue a este

    df = df[start:end]
    df = df.reset_index(drop=True)
    # print("start: %s, end: %s" % (start,end))
    return df

In [52]:
# !pip3 install html5lib
# !pip3 install bs4
# !pip3 install BeautifulSoup
# !pip3 install lxml

In [53]:
import requests
from bs4 import BeautifulSoup
from time import sleep
def search_link(df):
    soup = []   
    for link in df['links']:
        URL = link

        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        # Here the user agent is for Edge browser on windows 10. You can find your browser user agent from the above given link.
        r = requests.get(url=URL, headers=headers)
        # print(r.content)

        soup.append(BeautifulSoup(r.content, 'html5lib')) # If this line causes an error, run 'pip install html5lib' or install html5lib
        # print(soup[-1].prettify())
        sleep(40)

    return soup

In [54]:
def titles(soup):
    titles = []


    for s in soup:
        # title = s.find('h1', attrs = {"class": "Headline-headline-2FXIq Headline-black-OogpV ArticleHeader-headline-NlAqj"}) #for reuters

        title = s.find('h1', attrs = {"class": "artTitle font_faus"}) #for economictimes
        if title is not None:
            titles.append(title.text) 
        else:
            titles.append('HOLAXD')

    return titles

In [55]:
def bodies (soup):
    bodies = []

    for s in soup:
        body = ""

        # table = s.find("div", attrs = {"class": "TwoColumnsLayout-body-86gsE ArticlePage-body-container-10RhS"}) #for retuers

        table = s.find("div", attrs = {"class": "pageContent flt"}) #for economictimes
        
        if table is not None:
            # for row in table.findAll("p",
                                    # attrs = {"class":"Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x"}):  #for reuters
                # body += row.text + "\n"

            table = table.find("article", attrs = {"class": "artData clr"})

            if table is None:
                table = s.find("div", attrs = {"class": "pageContent flt"}).find("article", attrs = {"class": "artData clr paywall"})

            if table is not None:
                table = table.find("div", attrs = {"class": "artText"})

                if table is not None:
                    for div in table.find_all("div", {'class':"ar_wrp arwd_ld_chk font_mon clr hide"}): 
                        div.decompose()

                    bodies.append(table.text)
                else:
                    bodies.append('HOLAXD')

            else:
                bodies.append('HOLAXD')

        else:
            bodies.append('HOLAXD')

    return bodies

In [56]:
df['title'] = pd.Series(titles)
df['body'] = pd.Series(bodies)

In [57]:
import os
# if file does not exist write header 
def cargar_csv(df): 
   if not os.path.isfile('sentfin_extended22.csv'):
      df.to_csv('sentfin_extended22.csv', header='column_names', index=False)
   else: # else it exists so append without writing the header
      df.to_csv('sentfin_extended22.csv', mode='a', header=False, index=False)

   # df.to_csv("file.csv", encoding='utf-8', index=False)

In [58]:
import pandas as pd
import numpy as np
import pathlib as pl
import sklearn
import requests
from bs4 import BeautifulSoup
from time import sleep
start = 360
for end in range(380,500,20):
    path_is = get_path()
    # print("path_is = ", path_is)
    arreglo = arrange(path_is)
    # print("arreglo = ", arreglo)
    df = count(start, end,arreglo)
    # print("df = ", df)
    sopa = search_link (df)
    # print("sopa = ", sopa)
    titulos = titles(sopa)
    # print("titulos = ", titulos)
    df['title'] = pd.Series(titulos)
    df['body'] = pd.Series(bodies(sopa))
    # print ("partido:",df[start:end])
    # print("completo:",df)
    cargar_csv(df)
    print("se termino hasta: ",end)
    start = end

se termino hasta:  380
se termino hasta:  400
se termino hasta:  420
se termino hasta:  440
se termino hasta:  460
se termino hasta:  480
